## Dataset Construction

In [39]:
dir = 'data/full data/'

import json
import os
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

from utils import elem2sent

import warnings
warnings.filterwarnings('ignore')

In [4]:
meta_data_df = pd.read_excel('data/category_configuration_marked.xlsx', sheet_name = 'article_names_matching')
title2category = dict(zip(meta_data_df["Article Title"], meta_data_df["Category 2"]))

In [6]:
from unicodedata import category


def get_df(dir):
    cnt = 0
    df = pd.DataFrame(columns=['article', 'category'])
    
    for file_name in os.listdir(dir):
        try:
            root = ET.parse(dir + file_name).getroot()
            meta_data = root[0]
            doc_body = root[1][2]
        except Exception as e:
            continue

        all_articles_with_title = [_ for _ in doc_body.findall(".//div[@type='article']") if 'title' in _.attrib]

        for article in all_articles_with_title:
            title = article.attrib['title'].lower()
            sentences = ''.join(elem2sent(article))
            category = title2category.get(title, 'unknown')
            df = df.append({'article': sentences, 'title':title, 'category': category}, ignore_index=True)
            cnt += 1

    return df

df = get_df(dir)
df.to_csv('generated_data/category_assignment_full.csv', index=False)


In [7]:
df.head()

,article,category,title
0,If either Contracting Party makes payment to a...,unknown,s
1,Where a matter is governed both by this Conven...,unknown,article §
2,"For the purposes of this Agreement(a) ""investm...",Definition,definitions
3,(1) Each Contracting Party shall encourage and...,Promotion and Admission,promotion and protection of investment
4,(1) Neither Contracting Party shall in its ter...,Most-Favoured-Nation Treatment,most-favoured-nation provisions


### Data preprocessing

In [42]:
df_with_category = df[df['category'] != 'unknown']

#drop the title column
df_with_category = df_with_category.drop(columns=['title'])

#replace the category with less than 5 samples with 'others'
df_with_category['category'] = df_with_category['category'].apply(lambda x: 'others' if len(df_with_category[df_with_category['category'] == x]) < 5 else x)

#extract the category labels as a map
category2label = dict(zip(df_with_category['category'].unique(), range(len(df_with_category['category'].unique()))))
#save the map
with open('generated_data/category2label.json', 'w') as f:
    json.dump(category2label, f)

#replace the category with the label
df_with_category['category'] = df_with_category['category'].apply(lambda x: category2label[x])

#train test split
train_df, test_df = train_test_split(df_with_category, test_size=0.2, random_state=42)

#save the train and test data
train_df.to_csv('generated_data/category_train.csv', index=False)
test_df.to_csv('generated_data/category_test.csv', index=False)

## Test with a auto ml model as baseline

#### Load the data

In [43]:
train_df = pd.read_csv('generated_data/category_train.csv')
test_df = pd.read_csv('generated_data/category_test.csv')

In [46]:
from autogluon.text import TextPredictor

predictor = TextPredictor(
    label='category', 
    problem_type='multiclass',
    eval_metric='f1_micro', 
    path='./models'
)
predictor.fit(train_df, time_limit=None, presets='medium_quality_faster_train')

Global seed set to 123
Downloading: 100%|██████████| 29.0/29.0 [00:00<?, ?B/s]
Downloading: 100%|██████████| 665/665 [00:00<?, ?B/s] 
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 1.65MB/s]
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 1.99MB/s]
Downloading: 100%|██████████| 51.7M/51.7M [00:01<00:00, 48.9MB/s]
Auto select gpus: [0]
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                         | Params
-------------------------------------------------------------------
0 | model             | HFAutoModelForTextPrediction | 13.5 M
1 | validation_metric | Accuracy                     | 0     
2 | loss_func         | CrossEntropyLoss             | 0     
-------------------------------------------------------------------
13.5 M    Trainable params


Epoch 0:  50%|████▉     | 1877/3755 [01:46<01:46, 17.69it/s, loss=0.613, v_num=]

Epoch 0, global step 97: 'val_accuracy' reached 0.86640 (best 0.86640), saving model to 'D:\\x\\cs\\project\\AI4LEGAL\\models\\epoch=0-step=97.ckpt' as top 3


Epoch 0: 100%|█████████▉| 3754/3755 [03:25<00:00, 18.23it/s, loss=0.445, v_num=]

Epoch 0, global step 195: 'val_accuracy' reached 0.91640 (best 0.91640), saving model to 'D:\\x\\cs\\project\\AI4LEGAL\\models\\epoch=0-step=195.ckpt' as top 3


...

Epoch 9: 100%|█████████▉| 3754/3755 [03:25<00:00, 18.24it/s, loss=0.165, v_num=]  

Epoch 9, global step 1959: 'val_accuracy' reached 0.95160 (best 0.95160), saving model to 'D:\\x\\cs\\project\\AI4LEGAL\\models\\epoch=9-step=1959.ckpt' as top 3


Epoch 9: 100%|██████████| 3755/3755 [03:26<00:00, 18.16it/s, loss=0.16, v_num=] 


Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting DataLoader 0: 100%|██████████| 79/79 [00:03<00:00, 24.34it/s]


In [48]:
test_score = predictor.evaluate(test_df, metrics=['acc', 'f1_macro'])
print(test_score)

Predicting DataLoader 0: 100%|██████████| 216/216 [00:07<00:00, 28.44it/s]
{'acc': 0.9564206856478792, 'f1_macro': 0.8612385243420871}
